# Real VS Fake Faces Classification

## Evaluate your model

In [2]:
import os

from sklearn.metrics import accuracy_score, classification_report

import utils
import model

## Experiment settings

Set here the two variables in the following way:

- **CONFIG_YML:** assign the path to the config.yml file used for the experiment you want to evaluate
- **WEIGHTS:** assign the path to the model weights (.h5 file) you want to evaluate


In [5]:
# Modify this
CONFIG_YML = "config.yml"
WEIGHTS = "weights/model.better.h5"

In [6]:
# (!) Don't touch this cell, will load the config file specified before
config = utils.load_config(CONFIG_YML)
config

{'seed': 21,
 'data': {'directory': '/data/train/',
  'labels': 'inferred',
  'label_mode': 'categorical',
  'validation_split': 0.2,
  'image_size': [256, 256],
  'batch_size': 10},
 'model': {'weights': 'imagenet',
  'input_shape': [256, 256, 3],
  'classes': 2,
  'dropout_rate': 0.1,
  'data_aug_layer': {'random_flip': {'mode': 'horizontal'},
   'random_contrast': {'factor': 0.1}},
  'regularizer': 'l2',
  'trainable': True},
 'compile': {'optimizer': {'adam': {'learning_rate': 0.005,
    'epsilon': 0.05,
    'amsgrad': True}},
  'loss': 'binary_crossentropy',
  'metrics': ['categorical_accuracy']},
 'fit': {'epochs': 10,
  'callbacks': {'model_checkpoint': {'filepath': '/model/weights/model.better.h5',
    'save_best_only': True},
   'early_stopping': {'monitor': 'val_loss',
    'patience': 2,
    'restore_best_weights': True},
   'tensor_board': {'log_dir': '/model/weights/logs'}}}}

In [4]:
# (!) Don't touch this cell, will use the config file to infer the class names
#     and also to load the corresponding testing dataset.
#     If you get an error, you must review your data/code.

CLASSES = utils.get_class_names(config)

if len(CLASSES) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your data!"
    )

_dirname, _ = os.path.split(config['data']['directory'])
TEST_FOLDER = os.path.join(_dirname, 'test')

if not os.path.exists(TEST_FOLDER):
    raise ValueError("'test' folder not found!")
    
if len(os.listdir(TEST_FOLDER)) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your testing dataset!"
    )
    
if set(os.listdir(TEST_FOLDER)) != set(CLASSES):
    raise ValueError(
        "The name of the subfolders inside your test set "
        "doesn't match with the model classes!"
    )

## Load your model

Use `resnet_50.create_model()` and remember to properly setup the model weights!

Assign the model to the variable `cnn_model`.


In [5]:
# Complete this
# TODO
cnn_model = resnet_50.create_model(weights=WEIGHTS)

# It should print your model correctly
print(cnn_model.summary())

2022-06-22 16:09:06.887115: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-22 16:09:06.897440: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-22 16:09:06.898333: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-22 16:09:06.900729: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                             

## Get predictions from testing dataset

In [6]:
# (!) Don't touch this cell, will use the loaded model and
#     the function utils.predict_from_folder() to get 
#     model predictions and the corresponding true labels
#     so we can measure the accuracy

predictions, labels = utils.predict_from_folder(
    folder=TEST_FOLDER, 
    model=cnn_model, 
    input_size=config["data"]["image_size"], 
    class_names=CLASSES,
)

if len(predictions) != len(labels):
    raise ValueError(
        "The lenght of predictions and labels lists doesn't match!"
    )

if not isinstance(predictions[0], str):
    raise ValueError(
        "Model predictions should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )

if not isinstance(labels[0], str):
    raise ValueError(
        "Ground true labels should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )


2022-06-22 16:09:15.023896: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200
2022-06-22 16:09:15.357925: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-06-22 16:09:15.358669: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-06-22 16:09:15.358757: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-06-22 16:09:15.359726: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-06-22 16:09:15.359864: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


In [7]:
# (!) Don't touch this cell

print(classification_report(y_true=labels, y_pred=predictions))


                                                        precision    recall  f1-score   support

                            AM General Hummer SUV 2000       0.84      0.95      0.89        44
                             Acura Integra Type R 2001       0.79      0.93      0.85        44
                                   Acura RL Sedan 2012       0.72      0.81      0.76        32
                                   Acura TL Sedan 2012       0.76      0.98      0.86        43
                                  Acura TL Type-S 2008       0.91      0.95      0.93        42
                                  Acura TSX Sedan 2012       0.93      0.70      0.80        40
                              Acura ZDX Hatchback 2012       0.81      0.87      0.84        39
              Aston Martin V8 Vantage Convertible 2012       0.65      0.82      0.73        45
                    Aston Martin V8 Vantage Coupe 2012       0.85      0.71      0.77        41
                  Aston Martin Virage C

In [8]:
# (!) Don't touch this cell

acc = accuracy_score(y_true=labels, y_pred=predictions)

print(f"Your model accuracy is {acc:.4f}!")

if acc < .3:
    raise ValueError("Your model accuracy is too low :(\nYou can do it better! :)")


Your model accuracy is 0.8591!
